# Model Training

In [39]:
import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [40]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [41]:
batchSize = 128
numClass = 10
epochs = 12

img_rows, img_cols = 28, 28

In [42]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [43]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [44]:
y_train = keras.utils.to_categorical(y_train, numClass)
y_test = keras.utils.to_categorical(y_test, numClass)

In [45]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), \
                 activation='relu', input_shape=input_shape, \
                 padding='same', name='layer0'))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same', name='layer1'))
#model.add(Conv2D(20, (5,5), activation='relu', padding='same', name='layer2'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(numClass, activation='softmax'))

In [46]:
model.compile(loss=keras.losses.categorical_crossentropy, \
              optimizer=keras.optimizers.Adadelta(), \
              metrics=['accuracy'])

In [47]:
model.fit(x_train, y_train, batch_size=batchSize, epochs=epochs, \
          verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/12
469/469 [==============================] - 68s 145ms/step - loss: 2.2711 - accuracy: 0.1789 - val_loss: 2.2326 - val_accuracy: 0.4610
Epoch 2/12
469/469 [==============================] - 64s 137ms/step - loss: 2.2065 - accuracy: 0.3221 - val_loss: 2.1525 - val_accuracy: 0.5943
Epoch 3/12
469/469 [==============================] - 64s 137ms/step - loss: 2.1194 - accuracy: 0.4322 - val_loss: 2.0421 - val_accuracy: 0.6596
Epoch 4/12
469/469 [==============================] - 65s 138ms/step - loss: 2.0011 - accuracy: 0.5007 - val_loss: 1.8912 - val_accuracy: 0.6950
Epoch 5/12
469/469 [==============================] - 65s 138ms/step - loss: 1.8467 - accuracy: 0.5534 - val_loss: 1.6957 - val_accuracy: 0.7258
Epoch 6/12
469/469 [==============================] - 70s 149ms/step - loss: 1.6606 - accuracy: 0.5917 - val_loss: 1.4703 - val_accuracy: 0.7513
Epoch 7/12
469/469 [==============================] - 72s 154ms/step - loss: 1.4631 - accuracy: 0.6290 - val_loss: 1.2470 - val_ac

In [48]:
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  0.6584964394569397
Test accuracy:  0.8496999740600586


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer0 (Conv2D)              (None, 28, 28, 5)         130       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 5)         0         
_________________________________________________________________
layer1 (Conv2D)              (None, 14, 14, 5)         630       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 5)           0         
_________________________________________________________________
layer2 (Conv2D)              (None, 7, 7, 20)          2520      
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 20)          0         
_________________________________________________________________
flatten (Flatten)            (None, 980)               0

In [41]:
model.save('mnist_keras_model')

W0628 14:36:06.439682 140734809017792 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Predict

In [12]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
def predict(idx, verbose=False):
    img = x_test[idx]
    label = y_test[idx]

    predict = model.predict_proba(np.expand_dims(img, axis=0))
    if verbose:
        print(f'Index: {idx}')
        cnt = 0
        for i in predict[0]:
            print('[{}] {:.2%}'.format(cnt, i))
            cnt += 1
        print('Predicted: ', predict[0].argmax())
        print('Label: ', label.argmax())
        plt.imshow(np.reshape(img, [28, 28]),interpolation='nearest', cmap='gray')
    return (predict[0].argmax(), label.argmax())

In [19]:
# dictionary of mal-predicted indices 
wrong = {}

for i in range(x_test.shape[0]):
    pred = predict(i)
    if pred[0] != pred[1]:
        wrong[i] = (pred[0], pred[1])

W0628 14:15:20.282444 140734809017792 deprecation.py:323] From <ipython-input-13-283aa33df260>:5: Sequential.predict_proba (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use `model.predict()` instead.


In [43]:
#choice = []
choice = open('dropdown.txt', 'w')
for k, v in wrong.items():
    string = '%d) predicted: %d, label: %d' %(k, v[0], v[1])
    #choice.append(string)
    choice.write(string + '\n')